In [17]:
import os
import numpy as np
import cv2

def generate_result(input_image_path, output_image_path):
    prototxt_path = '../colorization/models/colorization_deploy_v2.prototxt'
    model_path = '../colorization/models/colorization_release_v2.caffemodel'
    kernel_path = '../colorization/models/pts_in_hull.npy'

    net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)
    points = np.load(kernel_path)

    points = points.transpose().reshape(2, 313, 1, 1)
    net.getLayer(net.getLayerId("class8_ab")).blobs = [points.astype(np.float32)]
    net.getLayer(net.getLayerId("conv8_313_rh")).blobs = [np.full([1, 313], 2.606, dtype="float32")]

    bw_image = cv2.imread(input_image_path)
    normalized = bw_image.astype("float32") / 255.0
    lab = cv2.cvtColor(normalized, cv2.COLOR_BGR2LAB)

    resized = cv2.resize(lab, (224, 224))
    L = cv2.split(resized)[0]
    L -= 50

    net.setInput(cv2.dnn.blobFromImage(L))
    ab = net.forward()[0, :, :, :].transpose((1, 2, 0))

    ab = cv2.resize(ab, (bw_image.shape[1], bw_image.shape[0]))
    L = cv2.split(lab)[0]

    colorized = np.concatenate((L[:, :, np.newaxis], ab), axis=2)
    colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
    colorized = (255.0 * colorized).astype("uint8")

    # Save the colorized image
    cv2.imwrite(output_image_path, colorized)

def colorize_images_in_directory(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith(".jpg"):
                relative_path = os.path.relpath(root, input_dir)
                input_path = os.path.join(root, file)
                output_path = os.path.join(output_dir, relative_path, file)
                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                generate_result(input_path, output_path)

# Specify input and output directories
input_dir = 'bw_output'
output_dir = 'color_output'

# Colorize images in the input directory and maintain the folder structure
colorize_images_in_directory(input_dir, output_dir)
